In [ ]:
from bertopic import BERTopic
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

nltk.download('stopwords')

In [2]:
data = pd.read_excel('amazon_dog_food_reviews_FASTopic_BERTopic.xlsx')
data = data.dropna()
data

,Review ID,time,Author Name,Author URL,Rating,Review Title,Review URL,text,Verified Purchase,clean_text
0,R1KVDJU6Z60P5L,31/12/2020,The Martin‚Äôs,https://www.amazon.com/gp/profile/amzn1.accoun...,1,Horrible food for dogs and makes them super ga...,https://www.amazon.com/gp/customer-reviews/R1K...,My dog ate it a couple times got super sick. I...,False,eat couple time super sick thought transition ...
1,R3BDTTWQEA8WVO,31/12/2020,Judy,https://www.amazon.com/gp/profile/amzn1.accoun...,5,My dog loves this,https://www.amazon.com/gp/customer-reviews/R3B...,My dog likes it and it is highly recommended.,True,like highly recommend
2,R2HIKEFD35S0UD,31/12/2020,repete,https://www.amazon.com/gp/profile/amzn1.accoun...,5,"Great dog food, tastes like chicken",https://www.amazon.com/gp/customer-reviews/R2H...,My dog has eaten this for years without issue....,True,eat year issue add water break treat heaven
3,R3CSCZS4YUDSIV,27/12/2020,Brian Moore,https://www.amazon.com/gp/profile/amzn1.accoun...,5,Great Dogfood and Service,https://www.amazon.com/gp/customer-reviews/R3C...,"Food showed up at my door, no carrying it from...",True,food door carry store love it
4,R21TC1KPX49VUY,26/12/2020,Lindsay Good,https://www.amazon.com/gp/profile/amzn1.accoun...,5,My dog loves it!,https://www.amazon.com/gp/customer-reviews/R21...,My dog loves it! This is one of the few brands...,True,love it brand eat issue offer small bite get g...
...,...,...,...,...,...,...,...,...,...,...
3686,R1G1DJ6CUIN7V8,01/06/2016,LadyKathy,https://www.amazon.com/gp/profile/amzn1.accoun...,5,Good food for the price,https://www.amazon.com/gp/customer-reviews/R1G...,"Good food for the price, and with auto shipmen...",True,good price auto shipment carry heavy bag store...
3687,R2TZ9YIF9TDCDE,01/05/2016,C. Crosley,https://www.amazon.com/gp/profile/amzn1.accoun...,5,I love their company,https://www.amazon.com/gp/customer-reviews/R2T...,Blue Buffalo has done it again. I love their c...,True,again love company productseverything allergic...
3688,R28WLUJG3QXETY,01/04/2016,ted smith,https://www.amazon.com/gp/profile/amzn1.accoun...,3,Three Stars,https://www.amazon.com/gp/customer-reviews/R28...,My dog eats the kibble but spits the life sour...,False,eat kibble spits life source bit floor
3689,R1XCS7H039UH5K,01/03/2016,Sherry P,https://www.amazon.com/gp/profile/amzn1.accoun...,4,All dogs seemed like they were starving by nwx...,https://www.amazon.com/gp/customer-reviews/R1X...,Dogs did well with it in regards to digestion....,True,dog digestion weimaraner lose bit weight tho i...


In [ ]:
# create a list of speeches
docs = data['text'].tolist()

In [5]:
# extended list of English stopwords

stopwords_extended = [
"0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about",
 "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually",
 "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards",
 "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost",
 "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst",
 "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone",
 "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear",
 "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't",
 "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au",
 "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1",
 "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes",
 "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins",
 "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between",
 "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br",
 "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca",
 "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce",
 "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly",
 "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently",
 "consider", "considering", "contain", "containing", "contains", "corresponding",
 "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's",
 "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de",
 "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't",
 "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't",
 "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3",
 "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either",
 "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough",
 "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu",
 "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex",
 "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi",
 "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl",
 "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth",
 "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further",
 "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give",
 "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten",
 "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly",
 "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd",
 "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres",
 "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself",
 "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr",
 "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid",
 "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm",
 "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed",
 "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead",
 "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it",
 "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj",
 "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know",
 "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter",
 "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's",
 "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look",
 "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly",
 "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile",
 "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml",
 "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu",
 "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely",
 "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn",
 "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine",
 "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor",
 "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt",
 "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often",
 "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one",
 "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others",
 "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over",
 "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount",
 "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per",
 "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po",
 "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present",
 "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt",
 "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran",
 "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref",
 "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl",
 "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn",
 "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say",
 "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem",
 "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously",
 "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's",
 "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side",
 "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl",
 "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something",
 "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically",
 "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly",
 "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure",
 "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td",
 "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that",
 "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves",
 "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein",
 "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've",
 "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv",
 "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though",
 "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus",
 "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top",
 "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying",
 "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d",
 "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike",
 "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used",
 "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va",
 "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol",
 "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was",
 "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll",
 "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what",
 "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's",
 "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon",
 "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever",
 "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi",
 "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder",
 "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www",
 "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv",
 "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your",
 "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt",
 "z", "zero", "zi", "zz"]

In [6]:
# create stopwords list
standard_stopwords = list(stopwords.words('english'))

# additiona tokens to remove
additional_stopwords = ['blue','buffalo','dog','food','ha','month','ago'] 

# combine standard, extended stopwords, and additional tokens
full_stopwords = standard_stopwords + additional_stopwords + stopwords_extended

# define tokenizer retrurning lemmatized text without numbers
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        doc = re.sub(r'\d+', '', doc)  # clean numbers
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)] # lemmatize

# vectorizer makes data processing and generates bigrams
vectorizer_model = CountVectorizer(tokenizer=LemmaTokenizer(),
                                  ngram_range=(2, 2),
                                  stop_words=full_stopwords)

# set-up model
model = BERTopic(n_gram_range=(2,2), # returns bigrams
                nr_topics=9,         # generate 9 topics, leave -1 for outliers
                top_n_words=20,      # return top 20 bigrams
                min_topic_size=20,   # each topic contains at least 20 tokens
                vectorizer_model=vectorizer_model,
                umap_model = UMAP(random_state=1))  # setting seed topics reproduce

# fit model to data
topics, probabilities = model.fit_transform(docs)

In [7]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1172,-1_brown rice_grain free_pet store_chicken brown,"[brown rice, grain free, pet store, chicken br...",[If it's in the budget doggie gets a decent br...
1,0,852,0_picky eater_absolutely love_love great_sensi...,"[picky eater, absolutely love, love great, sen...","[My sheltie always is a picky eater, but I tho..."
2,1,553,1_love love_love good_love great_love stuff,"[love love, love good, love great, love stuff,...","[My dogs love this stuff!, My dog loves this s..."
3,2,432,2_life protection_bag amazon_pet store_read re...,"[life protection, bag amazon, pet store, read ...",[Our dog almost died due to some kind of intox...
4,3,261,3_great product_good price_great price_product...,"[great product, good price, great price, produ...","[Great product, Great product., Great product ..."
5,4,257,4_bag ripped_box bag_inside box_hole bag,"[bag ripped, box bag, inside box, hole bag, op...","[Bag was ripped food everywhere, the bag came ..."
6,5,48,5_great great_great excellent_excellent great_...,"[great great, great excellent, excellent great...","[Great, Great, Great!!!]"
7,6,25,6_ama esta_esta comida_aburrian otras_pero esta,"[ama esta, esta comida, aburrian otras, pero e...","[Mi perrito ama esta comida, Fue una compra pa..."
8,7,21,7_small bite_small breed_bite easier_adult small,"[small bite, small breed, bite easier, adult s...",[My dog loves it so I love it too. The small ...


In [8]:
# Create dataframe with bigrams sorted by probabilities
import pandas as pd

# Retrieve the bigrams for each topic and select only the word columns
topic_0 = pd.DataFrame(model.get_topic(0), columns=["Topic_0_word", "Topic_0_prob"])[["Topic_0_word"]]
topic_1 = pd.DataFrame(model.get_topic(1), columns=["Topic_1_word", "Topic_1_prob"])[["Topic_1_word"]]
topic_2 = pd.DataFrame(model.get_topic(2), columns=["Topic_2_word", "Topic_2_prob"])[["Topic_2_word"]]
topic_3 = pd.DataFrame(model.get_topic(3), columns=["Topic_3_word", "Topic_3_prob"])[["Topic_3_word"]]
topic_4 = pd.DataFrame(model.get_topic(4), columns=["Topic_4_word", "Topic_4_prob"])[["Topic_4_word"]]
topic_5 = pd.DataFrame(model.get_topic(5), columns=["Topic_5_word", "Topic_5_prob"])[["Topic_5_word"]]
topic_6 = pd.DataFrame(model.get_topic(6), columns=["Topic_6_word", "Topic_6_prob"])[["Topic_6_word"]]
topic_7 = pd.DataFrame(model.get_topic(7), columns=["Topic_7_word", "Topic_7_prob"])[["Topic_7_word"]]

# # # Concatenate the DataFrames
topics_df = pd.concat([topic_0, topic_1, topic_2, topic_3, topic_4, topic_5, topic_6,topic_7], axis=1)
topics_df

,Topic_0_word,Topic_1_word,Topic_2_word,Topic_3_word,Topic_4_word,Topic_5_word,Topic_6_word,Topic_7_word
0,picky eater,love love,life protection,great product,bag ripped,great great,ama esta,small bite
1,absolutely love,love good,bag amazon,good price,box bag,great excellent,esta comida,small breed
2,love great,love great,pet store,great price,inside box,excellent great,aburrian otras,bite easier
3,sensitive stomach,love stuff,read review,product great,hole bag,nice great,pero esta,adult small
4,love picky,product love,healthy weight,good product,opened box,great nice,seca aburrian,love small
5,great price,love product,protection formula,fast delivery,bag hole,excellent woof,perrito ama,great small
6,picky love,great product,bought bag,buy great,ripped open,woof nice,tiene mi,bite small
7,love love,love healthy,year love,price great,bag bag,awesome woof,perra tiene,breed older
8,class action,pup love,reading review,product good,bag box,awesome awesome,toda necesidades,smaller bite
9,delivered door,love brand,buying year,excellent product,bag opened,great fine,traga bien,size bite


In [9]:
topics_df.to_excel('bigrams_bertopic_8_top20.xlsx', index=False)

In [10]:
# Topic activity over time
import plotly.graph_objects as go

# Create timestamps
data['time'] = pd.to_datetime(data['time'])
timestamps = data['time'].to_list()

# Generate the topics over time
topics_over_time = model.topics_over_time(docs, timestamps, nr_bins=20)

# Filter out topic -1 containing outliers
topics_over_time_filtered = topics_over_time[topics_over_time['Topic']  != -1]

# Visualize the filtered topics over time
fig = model.visualize_topics_over_time(topics_over_time_filtered)

# Update legend to display only the topic number
fig.data = sorted(fig.data, key=lambda trace: trace.name)

for trace in fig.data:
    trace.name = trace.name[0]

# Update the layout
fig.update_layout(
    width=1200,
    height=600,
    title='',
    legend_title_text='Topic',
    xaxis_tickangle=45         # Set x-axis labels to 45-degree angle
)

# Show the figure
fig.show()